# 骆驼先知

骆驼先知是 李鲁鲁 受到吴恩达的prompt Engineering的启发

制作的一个纪伯伦的《先知》的拓展版本

运行这个notebook你需要OpenAI的API Token

项目链接 [https://github.com/LC1332/Prophet-Andrew-Ng](https://github.com/LC1332/Prophet-Andrew-Ng)

骆驼先知是[Luotuo(骆驼)](https://github.com/LC1332/Luotuo-Chinese-LLM)的子项目之一，后者由李鲁鲁，冷子昂，陈启源发起。

## Gradio版本的说明

Gradio版本是在Retrieve版本上增加的交互界面

使用LuotuoBERT去搜索最接近的文本，然后进行In Context的few shot Learning

在colab运行时，我们建议使用GPU来进行运行。

In [1]:
! pip install openai
! pip install gradio
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 19.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 84.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.1/288.1 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# !rm -r -f /content/Prophet-Andrew-Ng/
#从项目中获取数据
!git clone https://github.com/LC1332/Prophet-Andrew-Ng

Cloning into 'Prophet-Andrew-Ng'...
remote: Enumerating objects: 337, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 337 (delta 40), reused 47 (delta 34), pack-reused 272
Receiving objects: 100% (337/337), 1.44 MiB | 14.77 MiB/s, done.
Resolving deltas: 100% (163/163), done.


## 数据读取

读取prompt-data中的文本数据，作为in-context-learning的数据

In [3]:
# 如果你使用本地的版本，你的路径应该为
# prophet_data_folder = './../prophet-data'

# 在这里我们考虑colab的版本
prophet_data_folder = '/content/Prophet-Andrew-Ng/prophet-data'

import os

titles = []
title_to_text = {}

# scan all txt file in prophet_data_folder
for file in os.listdir(prophet_data_folder):
    if file.endswith('.txt'):
        title_name = file[:-4]
        titles.append(title_name)

        with open(os.path.join(prophet_data_folder, file), 'r') as f:
            title_to_text[title_name] = f.read()

# report length of each text
for title in titles:
    print(title, len(title_to_text[title]))

爱 789
时间 335
祈祷 618
善与恶 719
友谊 466
理智与热情 508
孩子 315
传授 286
快乐 851
罪与罚 1214
死亡 403
自由 781
美 679
痛苦 289
衣服 256
法律 640
悲伤与欢乐 388
房舍 859
施舍 874
宗教 582
买卖 410
饮食 423
自知之明 309
婚姻 292
劳作 935
说话 357


我们将在后续课程中深入探究 OpenAI 提供的 ChatCompletion API 的使用方法，在此处，我们先将它封装成一个函数，你无需知道其内部机理，仅需知道调用该函数输入 Prompt 其将会给出对应的 Completion 即可。

In [4]:
import openai
# 一个封装 OpenAI 接口的函数，参数为 Prompt，返回对应结果
def get_completion(prompt, model="gpt-3.5-turbo"):
    '''
    prompt: 对应的提示
    model: 调用的模型，默认为 gpt-3.5-turbo(ChatGPT)，有内测资格的用户可以选择 gpt-4
    '''
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # 模型输出的温度系数，控制输出的随机程度
    )
    # 调用 OpenAI 的 ChatCompletion 接口
    return response.choices[0].message["content"]


定义先知和市民的身份，这里是为了方便后面的泛化

In [5]:
prophet_name = "先知"

citizen_name = "市民"

形成一个shot的句子输出

In [6]:
# 实现一个Python函数 ensemble_one_shot ，输入为prophet_name, citizen_name, sample_topic，sample_answer， 输出为一个组织好的字符串
# 例子输入: 先知, 市民, 衣服， "你们的衣服遮掩了你们许多的美，却不能遮盖住丑。 \n 尽管你们借衣服寻求隐私的自由，但你们找到的却是羁绊和束缚。"
# 例子输出: """
# <市民>:请和我们讨论一下"衣服"

# <先知>: 你们的衣服遮掩了你们许多的美，却不能遮盖住丑。
# 尽管你们借衣服寻求隐私的自由，但你们找到的却是羁绊和束缚。
# """
def ensemble_one_shot(prophet_name, citizen_name, sample_topic, sample_answer):
    # 组织对话文本
    dialogue = "<{}>:```请和我们讨论一下\"{}\"```\n\n<{}>:```{}```\n\n".format(citizen_name, sample_topic, prophet_name, sample_answer)
    return dialogue

# unit test for ensemble_one_shot
# prophet_name = "先知"
# citizen_name = "市民"
# sample_topic = "衣服"
# sample_answer = "你们的衣服遮掩了你们许多的美，却不能遮盖住丑。\n尽管你们借衣服寻求隐私的自由，但你们找到的却是羁绊和束缚。"

# dialogue = ensemble_one_shot(prophet_name, citizen_name, sample_topic, sample_answer)
# print(dialogue)

下面我们来组织完整的prompt，我们假设已经选取了两个主题作为例子 孩子和爱

In [7]:
# selected_sample = ["孩子","爱"]

def organize_prompt( query_topic, selected_sample ):
    prompt = """你的任务是以纪伯伦中《先知》的语言风格回答问题。\
    先知会使用大量的比喻修辞手法。感情色彩浓厚，富有感染力和感召力，具有启迪性和哲理性。\
    语言风格简洁明了，比喻和象征性的意象丰富多彩，既充满了哲理性和哲学深度，又富有感人肺腑的情感色彩。\n\n"""
    
    for sample_topic in selected_sample:
        # find sample_answer in dictionary
        sample_answer = title_to_text[sample_topic]
        prompt += ensemble_one_shot(prophet_name, citizen_name, sample_topic, sample_answer)

    prompt += """<{}>:```请和我们讨论一下"{}"```\n\n""".format(citizen_name, query_topic)

    return prompt

# write a unit test for organize_prompt
# query_topic = "时间"
# selected_sample = ["孩子","爱"]
# prompt = organize_prompt( query_topic, selected_sample )
# print(prompt)

## 升级randome_select到retrieve topics

这里我们开始引入LuotuoBERT，相应的项目见 [骆驼嵌入](https://github.com/LC1332/Luotuo-Text-Embedding)

我们需要先安装hugging face的代码库

然后从hugging face上，载入对应的模型

In [8]:
import torch
from scipy.spatial.distance import cosine
from transformers import AutoModel, AutoTokenizer
from argparse import Namespace
# Import our models. The package will take care of downloading the models automatically
tokenizer = AutoTokenizer.from_pretrained("silk-road/luotuo-bert")
model_args = Namespace(do_mlm=None, pooler_type="cls", temp=0.05, mlp_only_train=False, init_embeddings_model=None)
model = AutoModel.from_pretrained("silk-road/luotuo-bert", trust_remote_code=True, model_args=model_args)

A new version of the following files was downloaded from https://huggingface.co/silk-road/luotuo-bert:
- models.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


编写embedding函数

In [9]:
def get_embedding(text):
    if len(text) > 512:
        text = text[:512]
    texts = [text]
    # Tokenize the text
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    # Extract the embeddings
    # Get the embeddings
    with torch.no_grad():
        embeddings = model(**inputs, output_hidden_states=True, return_dict=True, sent_emb=True).pooler_output

    return embeddings[0]

存储两个list，embeddings和embed_to_title, 记录title和text到embedding

In [10]:
embeddings = []
embed_to_title = []

for title in titles:
    text = title_to_text[title]

    # divide text with \n\n
    divided_texts = text.split('\n\n')

    for divided_text in divided_texts:
        embed = get_embedding(divided_text)
        embeddings.append(embed)
        embed_to_title.append(title)
    
    embed_title = get_embedding(title)
    embeddings.append( embed )
    embed_to_title.append(title)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


定义similarity函数

In [11]:
def get_cosine_similarity( embed1 , embed2 ):
    return torch.nn.functional.cosine_similarity( embed1, embed2 , dim=0)

实现搜索函数

In [12]:
# implement retrieve_title function, return top k titles
def retrieve_title( query_embed, embeddings, embed_to_title, k ):
    # compute cosine similarity between query_embed and embeddings
    cosine_similarities = []
    for embed in embeddings:
        cosine_similarities.append( get_cosine_similarity( query_embed, embed ) )
    
    # sort cosine similarity
    sorted_cosine_similarities = sorted( cosine_similarities, reverse=True )

    top_k_index = []
    top_k_title = []

    for i in range(len(sorted_cosine_similarities)):
        current_title = embed_to_title[ cosine_similarities.index( sorted_cosine_similarities[i] ) ]
        if current_title not in top_k_title:
            top_k_title.append( current_title )
            top_k_index.append( cosine_similarities.index( sorted_cosine_similarities[i] ) )

        if len(top_k_title) == k:
            break
    
    return top_k_title

运行以后点击出现的链接就可以玩了

In [13]:
import gradio as gr
import openai


last_key = ''
openai.api_key = ''


def get_output(api_key, query_topic):
    if last_key != api_key and len(api_key)>5:
        openai.api_key = api_key
    query_embed = get_embedding(query_topic)
    selected_sample = retrieve_title(query_embed, embeddings, embed_to_title, 3)
    prompt = organize_prompt(query_topic, selected_sample)
    response = get_completion(prompt)
    return selected_sample, response

api_key_input = gr.inputs.Textbox(label="OpenAI API Token")
query_topic_input = gr.inputs.Textbox(label="你想和先知讨论的主题")

output1 = gr.outputs.Textbox(label="辅助例子")
output2 = gr.outputs.Textbox(label="模型输出")

interface = gr.Interface(fn=get_output,
                         inputs=[api_key_input, query_topic_input],
                         outputs=[output1, output2],
                         title="骆驼先知——模仿纪伯伦《先知》的哲学讨论",
                         description="Get the assistance examples and model output for a given topic",
                         allow_flagging=False,
                         theme="default")

if __name__ == "__main__":
    interface.launch(share = True)


/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:30: UserWarning: `optional` parameter is deprecated, and it has no effect
  super().__init__(
/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:30: UserWarning: `numeric` parameter is deprecated, and it has no effect
  super().__init__(
/usr/local/lib/python3.10/dist-packages/gradio/outputs.py:22: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/interface.py:323: UserWarning: The `allow_flagging` parameter in `Interface` nowtakes a string value ('auto', 'manual', or 'never'), not a boolean. Setting parameter to: 'never'.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://4ba6fb3c8eea5e80b2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


这里点击上面出现的链接就可以玩了

## 如何关闭

把对应PID的进程杀掉就可以

In [14]:
!ps aux | grep gradio

root        1601  0.0  0.0 723476 13080 ?        Sl   02:17   0:00 /usr/local/lib/python3.10/dist-packages/gradio/frpc_linux_amd64_v0.2 http -n hwNs5Y9zX7ccgsGtYB1Q29YAnpoqheHFAr0d7KGlIh8 -l 7860 -i 127.0.0.1 --uc --sd random --ue --server_addr 18.237.145.165:7000 --disable_log_color
root        1611  0.0  0.0   6904  3352 ?        S    02:17   0:00 /bin/bash -c ps aux | grep gradio
root        1613  0.0  0.0   6444   660 ?        S    02:17   0:00 grep gradio


In [15]:
!kill -9 3254 4171 4173

/bin/bash: line 0: kill: (3254) - No such process
/bin/bash: line 0: kill: (4171) - No such process
/bin/bash: line 0: kill: (4173) - No such process


集成所有的代码！

In [16]:
query_topic = "代码开源"
query_embed = get_embedding(query_topic)
selected_sample = retrieve_title( query_embed, embeddings, embed_to_title, 2 )
print('辅助sample:', selected_sample)
prompt = organize_prompt( query_topic, selected_sample )
response = get_completion(prompt)
print(response)

辅助sample: ['房舍', '爱']


AuthenticationError: ignored

In [ ]:
query_topic = "志愿者"
query_embed = get_embedding(query_topic)
selected_sample = retrieve_title( query_embed, embeddings, embed_to_title, 2 )
print('辅助sample:', selected_sample)
prompt = organize_prompt( query_topic, selected_sample )
response = get_completion(prompt)
print(response)

In [ ]:
query_topic = "童年"
query_embed = get_embedding(query_topic)
selected_sample = retrieve_title( query_embed, embeddings, embed_to_title, 2 )
print('辅助sample:', selected_sample)
prompt = organize_prompt( query_topic, selected_sample )
response = get_completion(prompt)
print(response)

In [ ]:
query_topic = "离婚"
query_embed = get_embedding(query_topic)
selected_sample = retrieve_title( query_embed, embeddings, embed_to_title, 2 )
print('辅助sample:', selected_sample)
prompt = organize_prompt( query_topic, selected_sample )
response = get_completion(prompt)
print(response)

In [ ]:
query_topic = "加班"
query_embed = get_embedding(query_topic)
selected_sample = retrieve_title( query_embed, embeddings, embed_to_title, 2 )
print('辅助sample:', selected_sample)
prompt = organize_prompt( query_topic, selected_sample )
response = get_completion(prompt)
print(response)

In [ ]:
query_topic = "买房还是租房"
query_embed = get_embedding(query_topic)
selected_sample = retrieve_title( query_embed, embeddings, embed_to_title, 2 )
print('辅助sample:', selected_sample)
prompt = organize_prompt( query_topic, selected_sample )
response = get_completion(prompt)
print(response)

In [ ]:
query_topic = "读博还是直接工作"
query_embed = get_embedding(query_topic)
selected_sample = retrieve_title( query_embed, embeddings, embed_to_title, 2 )
print('辅助sample:', selected_sample)
prompt = organize_prompt( query_topic, selected_sample )
response = get_completion(prompt)
print(response)

In [ ]:
query_topic = "加入大公司还是创业公司"
query_embed = get_embedding(query_topic)
selected_sample = retrieve_title( query_embed, embeddings, embed_to_title, 2 )
print('辅助sample:', selected_sample)
prompt = organize_prompt( query_topic, selected_sample )
response = get_completion(prompt)
print(response)

In [ ]:
query_topic = "内卷(在中国网络文化中，“内卷”这一词出现了语义偏移，用来指代内部恶性竞争、过度竞争的现象。如一家公司，原本实行八小时工作制，但有些人自愿加班，并得到管理层的赏识。此时，原先按时上下班的人开始担心自己成为劣势者，也自愿加班，久而久之，加班便成为常态，最后变成如果不自愿加班，就会影响自己在职场的生存，降低谈判力)"
query_embed = get_embedding(query_topic)
selected_sample = retrieve_title( query_embed, embeddings, embed_to_title, 2 )
print('辅助sample:', selected_sample)
prompt = organize_prompt( query_topic, selected_sample )
response = get_completion(prompt)
print(response)

In [ ]:
query_topic = "幼儿园"
query_embed = get_embedding(query_topic)
selected_sample = retrieve_title( query_embed, embeddings, embed_to_title, 2 )
print('辅助sample:', selected_sample)
prompt = organize_prompt( query_topic, selected_sample )
response = get_completion(prompt)
print(response)

In [ ]:
query_topic = "退休"
query_embed = get_embedding(query_topic)
selected_sample = retrieve_title( query_embed, embeddings, embed_to_title, 2 )
print('辅助sample:', selected_sample)
prompt = organize_prompt( query_topic, selected_sample )
response = get_completion(prompt)
print(response)

In [ ]:
query_topic = "早教(指在学龄前对儿童进行积极的教育)"
query_embed = get_embedding(query_topic)
selected_sample = retrieve_title( query_embed, embeddings, embed_to_title, 2 )
print('辅助sample:', selected_sample)
prompt = organize_prompt( query_topic, selected_sample )
response = get_completion(prompt)
print(response)

后续

下个版本做个gradio吧

- [x] 增加更好的前置提示词（提前总结先知的文风）
- [x] 补充完整的prophet data（一共20+个，现在只有5个）
- [x] 使用luotuoBERT索引更相关的主题
- [x] 做一个gradio的前端